In [6]:
################
#   ZELLE 01   #
################

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import LeaveOneOut, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, LogisticRegression
from sklearn.metrics import mean_absolute_error, r2_score, roc_auc_score, accuracy_score


In [7]:
################
#   ZELLE 02   #
################

excel_path = Path("Auswertung_Studie_Nutzungsform_aktualisiert.xlsx")  # ggf. Pfad anpassen
assert excel_path.exists(), f"Datei nicht gefunden: {excel_path.resolve()}"

df_raw = pd.read_excel(excel_path, sheet_name="Final", header=[0, 1])

def flatten_cols(cols):
    out = []
    for a, b in cols:
        a = "" if "Unnamed" in str(a) else str(a).strip()
        b = "" if "Unnamed" in str(b) else str(b).strip()
        name = "_".join([x for x in (a, b) if x])
        out.append(name)
    return out

df = df_raw.copy()
df.columns = flatten_cols(df.columns)

df.head(20)


,Proband Nr.,Ben. Zeit,Kniewinkel 1,Kniewinkel 2,Alter,Geschlecht_weiblich,Geschlecht_männlich,Größe,Beininnenlänge,Sitzhöhe,...,Nutzung Methode_Einstellung durch Laden / Mechaniker,Nutzung Methode_Augenmaß / Gefühl,Nutzung Methode_Hüfte,Gefühl Sitzposition_Deutlich schlechter,Gefühl Sitzposition_Etwas schlechter,Gefühl Sitzposition_Kein Unterschied,Gefühl Sitzposition_Etwas besser,Gefühl Sitzposition_Deutlich besser,Längerer Zeitraum,Veränderungsimpuls
0,25,00:00:35.910000,67.7,157.3,27,x,NaN,164,75.5,70.0,...,NaN,x,NaN,NaN,x,NaN,NaN,NaN,7,1
1,24,00:00:57.040000,68.2,143.1,26,NaN,x,200,92.0,80.0,...,NaN,x,NaN,NaN,NaN,x,NaN,NaN,8,1
2,23,00:01:10.400000,75.4,150.6,23,x,NaN,166,79.6,63.5,...,NaN,x,x,NaN,x,NaN,NaN,NaN,2,9
3,22,00:01:15.850000,72.4,161.0,23,x,NaN,172,82.9,71.0,...,NaN,x,NaN,NaN,NaN,x,NaN,NaN,9,1
4,21,00:02:19.390000,76.2,160.7,26,x,NaN,178,87.0,76.0,...,NaN,x,NaN,NaN,NaN,x,NaN,NaN,10,0
5,20,00:01:39.560000,86.4,148.6,26,NaN,x,193,92.0,77.0,...,NaN,NaN,NaN,NaN,NaN,x,NaN,NaN,10,0
6,19,00:01:14.580000,70.8,156.0,21,NaN,x,186,88.0,75.5,...,NaN,x,NaN,NaN,NaN,x,NaN,NaN,8,2
7,18,00:02:56.150000,76.6,143.2,26,NaN,x,194,91.0,75.5,...,NaN,x,NaN,NaN,NaN,x,NaN,NaN,8,4
8,17,00:02:17.640000,72.6,144.5,27,NaN,x,180,82.0,70.5,...,NaN,x,NaN,NaN,NaN,x,NaN,NaN,10,0
9,16,00:01:46.760000,81.3,147.4,23,x,NaN,193,88.0,80.5,...,NaN,x,NaN,NaN,NaN,x,NaN,NaN,10,0


In [8]:
################
#   ZELLE 03   #
################


# nur echte Datensätze behalten
df = df[df["Proband Nr."].notna()].copy()

# 1) 'x' -> 1 (nur wo es vorkommt)
df = df.replace({"x": 1})

# 2) nur object-Spalten versuchen zu numerifizieren (ohne errors="ignore")
obj_cols = [c for c in df.columns if c != "Proband Nr." and df[c].dtype == "object"]
for c in obj_cols:
    try:
        df[c] = pd.to_numeric(df[c])
    except (ValueError, TypeError):
        # bleibt Text, falls wirklich Text vorhanden ist
        pass

# 3) Ben. Zeit (falls vorhanden) robust in Sekunden umrechnen
if "Ben. Zeit" in df.columns:
    df["Ben. Zeit"] = pd.to_timedelta(df["Ben. Zeit"].astype(str), errors="coerce").dt.total_seconds()

# 4) fehlende Werte auffüllen (nach Konvertierung!)
df = df.fillna(0)

# optional: falls pandas noch object dtypes hat, sauber inferieren
df = df.infer_objects(copy=False)

df.shape
df.head(3)


C:\Users\mweiler\AppData\Local\Temp\ipykernel_23896\1127331608.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"x": 1})


,Proband Nr.,Ben. Zeit,Kniewinkel 1,Kniewinkel 2,Alter,Geschlecht_weiblich,Geschlecht_männlich,Größe,Beininnenlänge,Sitzhöhe,...,Nutzung Methode_Einstellung durch Laden / Mechaniker,Nutzung Methode_Augenmaß / Gefühl,Nutzung Methode_Hüfte,Gefühl Sitzposition_Deutlich schlechter,Gefühl Sitzposition_Etwas schlechter,Gefühl Sitzposition_Kein Unterschied,Gefühl Sitzposition_Etwas besser,Gefühl Sitzposition_Deutlich besser,Längerer Zeitraum,Veränderungsimpuls
0,25,35.91,67.7,157.3,27,1.0,0.0,164,75.5,70.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,7,1
1,24,57.04,68.2,143.1,26,0.0,1.0,200,92.0,80.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8,1
2,23,70.40,75.4,150.6,23,1.0,0.0,166,79.6,63.5,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,2,9


In [9]:

################
#   ZELLE 05   #
################


# Zielwert (Mitte des 150–155 Fensters)
TARGET = 152.5

# mehrere Fenster, damit du Sensitivität berichten kannst
WINDOWS = {
    "w150_155": (150.0, 155.0),
    "w145_155": (145.0, 155.0),
    "w145_160": (145.0, 160.0),
}

# Haupt-Outcomes
df["AbsDev_Target"] = (df["Kniewinkel 2"] - TARGET).abs()

# Abweichung zum Fenster (0 wenn drin, sonst Distanz zum nächsten Rand, mit Vorzeichen optional)
def window_deviation(x, lo, hi):
    # signed deviation to nearest boundary (0 inside)
    if x < lo: return x - lo
    if x > hi: return x - hi
    return 0.0

for key, (lo, hi) in WINDOWS.items():
    df[f"In_{key}"] = ((df["Kniewinkel 2"] >= lo) & (df["Kniewinkel 2"] <= hi)).astype(int)
    df[f"WinDev_{key}"] = df["Kniewinkel 2"].apply(lambda v: window_deviation(v, lo, hi))
    df[f"AbsWinDev_{key}"] = df[f"WinDev_{key}"].abs()

df[["Kniewinkel 2", "AbsDev_Target"] + [f"In_{k}" for k in WINDOWS]].head()


,Kniewinkel 2,AbsDev_Target,In_w150_155,In_w145_155,In_w145_160
0,157.3,4.8,0,0,1
1,143.1,9.4,0,0,0
2,150.6,1.9,1,1,1
3,161.0,8.5,0,0,0
4,160.7,8.2,0,0,0
